# Install and load necesary packages

In [1]:
# Please don't change this cell

import pandas as pd
import numpy as np  

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Please don't change this cell
df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Split dataset
## Random Train and Test Split

In [3]:
# please do not change this cell

from sklearn.model_selection import train_test_split

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 10)
train_df, test_df

# Training Dataset
trainsize = 0
train_ds = np.zeros((n_users, n_items))
item_popularity = np.zeros(n_items)
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
    item_popularity[row[2]-1] =  item_popularity[row[2]-1] + 1
    trainsize = trainsize + 1
#train_ds = pd.DataFrame(train_ds)

# Testing Dataset
testsize = 0
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    if item_popularity[row[2]-1] > 30:
        test_ds[row[1]-1, row[2]-1] = row[3]
        testsize = testsize + 1
#test_ds = pd.DataFrame(test_ds)

print("Construct the rating matrix based on train_df:")
print(train_ds)

print("Construct the rating matrix based on test_df:")
print(test_ds)

print("Testsize = " + str(testsize))
print("Trainsizse = " + str(trainsize))

943 users
1682 items
Construct the rating matrix based on train_df:
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
Construct the rating matrix based on test_df:
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testsize = 17678
Trainsizse = 80000


# Utils

In [4]:
# Please don't change this cell
# you can use this devaluate Utils here, and you can also implement your own MAE and RMSE calculation. 

EPSILON = 1e-9

def evaluate(test_ds, predicted_ds):
    '''
    Function for evaluating on MAE and RMSE
    '''
    # MAE
    mask_test_ds = test_ds > 0
    MAE = np.sum(np.abs(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32))

    # RMSE
    RMSE = np.sqrt(np.sum(np.square(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32)))

    return MAE, RMSE

# Your Solution

In [5]:
# Write your code here
# You are required to implement the required solution here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).

## User-based

In [6]:
train_ds

array([[0., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [7]:
# Write your code here
# You are required to implement the required solution here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).

### Compute Ajusted Euclidean Distance for Each Pair of Users in Training Dataset

In [8]:
# the maximum value for the recommendation system
V_max = np.max(train_ds)

# the minimum value for the recommendation system
V_min = np.min(train_ds)

In [9]:
GAMMA = 30
EPSILON = 1e-9

np_user_adjusted_euclidean_dist = np.zeros((n_users, n_users))

for u, user_u_vec in enumerate(train_ds):
    for v, user_v_vec in enumerate(train_ds):

        # ratings corated by the current pair of users
        mask_u = user_u_vec > 0
        mask_v = user_v_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_u), np.where(mask_v))
        if len(corrated_index) == 0:
            continue 
        
        # compute adjusted euclidean distance
        
        #ru,s - rv,s
        user_u_user_v_sub_dif = user_u_vec[corrated_index] - user_v_vec[corrated_index]
        
        # computes the square of the difference between the ratings of users on the 
        # co-rated items
        user_u_user_v_sub_dif_sq = np.square(user_u_user_v_sub_dif)
        
        # dist_u_v
        dist_u_v = np.sqrt(np.sum(user_u_user_v_sub_dif_sq))   
    
        # m is the number of members in the set of items co-rated by both users
        m = len(corrated_index)
            
        # V_max - V_min
        V_max_V_min_dif = V_max - V_min
        
        # It computes the square of the difference between the maximum and 
        # the minimum ratings for the recommendation system
        V_max_V_min_dif_sq = np.square(V_max_V_min_dif)

        # The product of the number of members in the set of all items co-rated
        # by both users and the square of the difference between the V_max and V_min
        V_max_V_min_dif_times_m = m * V_max_V_min_dif_sq
        
        # It gives the dist_max value
        dist_max = np.sqrt(V_max_V_min_dif_times_m)

        # dist_u_v / dist_max
        dist_uv_dist_max_quo = dist_u_v / dist_max
        
        # similarity
        sim = 1 - dist_uv_dist_max_quo

        np_user_adjusted_euclidean_dist[u][v] = sim

np_user_adjusted_euclidean_dist

array([[1.        , 0.77889168, 0.5527864 , ..., 0.78091098, 0.65358984,
        0.68434772],
       [0.77889168, 1.        , 0.65358984, ..., 0.61270167, 0.7018576 ,
        0.88452995],
       [0.5527864 , 0.65358984, 1.        , ..., 0.68377223, 0.6072078 ,
        1.        ],
       ...,
       [0.78091098, 0.61270167, 0.68377223, ..., 1.        , 0.8       ,
        0.74701779],
       [0.65358984, 0.7018576 , 0.6072078 , ..., 0.8       , 1.        ,
        0.76466064],
       [0.68434772, 0.88452995, 1.        , ..., 0.74701779, 0.76466064,
        1.        ]])

## Predict Rating

### Determination of the suitable K value

In [10]:
def find_best_k(train_ds, test_ds, np_user_adjusted_euclidean_dist, K_values):
    n_users, n_items = test_ds.shape
    EPSILON = 1e-9

    # Precompute sums and counts for all users to avoid repeated computation
    train_ds_sum = np.sum(train_ds, axis=1)
    train_ds_count = np.clip(train_ds, 0, 1).sum(axis=1) + EPSILON
    
    best_k = None
    best_mae = float('inf')
    best_rmse = float('inf')

    for K in K_values:
        np_predictions_ead = np.zeros((n_users, n_items))
        
        for (i, j), rating in np.ndenumerate(test_ds):
            if rating > 0:
                # Find top-k most similar users to the current user, excluding itself
                sim_user_ids = np.argsort(np_user_adjusted_euclidean_dist[i])[-(K + 1):-1]

                # The AED values of similar users
                sim_val = np_user_adjusted_euclidean_dist[i][sim_user_ids]

                # The similar users' ratings
                sim_users = test_ds[sim_user_ids]

                # Select the similar users who rated item j
                mask_rated_j = sim_users[:, j] > 0

                # If there is no similar users who have rated item s
                if not mask_rated_j.any():
                    # It computes the mean value of ratings of current user on the rated items
                    user_mean_rating = train_ds_sum[i] / train_ds_count[i]
                    np_predictions_ead[i][j] = user_mean_rating
                    np_predictions_ead[i][j] = np.clip(np_predictions_ead[i][j], 0, 5)

                # If there exist similar users who have rated item s
                else:
                    # Initialize set and lists for users with dist(c, c') = 0
                    sim_user_mask = sim_users[mask_rated_j]
                    sim_val_mask = sim_val[mask_rated_j]
                    C_cap_0 = np.where(sim_val_mask == 1)[0]

                    # If the C_cap_0 has no elements
                    if C_cap_0.size == 0:
                        # Lets find the absolute value of each current, similar users pairs
                        abs_sim_val = np.abs(sim_val_mask)
        #                 print(f"sim_val_mask: {sim_val_mask}")
                        # The sum of the absolute similarity values of each current, similar users pairs
                        abs_sim_val_sum = np.sum(abs_sim_val)

                        # Lets compute the value of k
                        k = 1 / abs_sim_val_sum

                        # The product of similarity and the rating of the similar_user on item j
                        sv_r_cprime_s_prod = sim_val_mask * sim_user_mask[:, j]

                        # The sum of product of similarity and rating of the similar_user on item j
                        sv_r_cprime_s_prod_sum = np.sum(sv_r_cprime_s_prod)

                        # Predictions using aed
                        np_predictions_ead[i][j] = k * sv_r_cprime_s_prod_sum
                        np_predictions_ead[i][j] = np.clip(np_predictions_ead[i][j], 0, 5)

                    else:
                        # Lets compute the number of co-rated items for each similar user
                        # Which has the dist(c, c_cap) = 0
                        m_ccap = np.array([
                            np.sum(np.logical_and(su > 0, test_ds[i] > 0))
                            for su in sim_user_mask[C_cap_0]
                        ])
                        
                        # Calculates the rating of similar user on item j
                        r_ccap_s = sim_user_mask[C_cap_0, j]

                        # Calculates the sum of the ratings of similar users on item j
                        m_ccap_sum = np.sum(m_ccap)

                        # Computes the value for K note
                        k_0 = 1 / m_ccap_sum

                        # Computes the product of number of co-rated itmes for user c and ccap and 
                        # the ratings of the corresponding ccap
                        m_ccap_ccap_s_prod = m_ccap * r_ccap_s

                        # The total sum of the product of m_ccap and r_ccap_s
                        m_ccap_ccap_s_prod_sum = np.sum(m_ccap_ccap_s_prod)

                        # the predicted rating
                        np_predictions_ead[i][j] = k_0 * m_ccap_ccap_s_prod_sum
                        np_predictions_ead[i][j] = np.clip(np_predictions_ead[i][j], 0, 5)
        
        mae, rmse = evaluate(np_predictions_ead, test_ds)
        print(f"K: {K}, MAE: {mae}, RMSE: {rmse}")

        if mae < best_mae and rmse < best_rmse:
            best_mae = mae
            best_rmse = rmse
            best_k = K

    return best_k, best_mae, best_rmse

# Example usage
K_values = range(10, 101, 5)  # Try K values from 10 to 50 in steps of 10
best_k, best_mae, best_rmse = find_best_k(train_ds, test_ds, np_user_adjusted_euclidean_dist, K_values)
print(f"Best K: {best_k}, Best MAE: {best_mae}, Best RMSE: {best_rmse}")


K: 10, MAE: 0.4024433403050918, RMSE: 0.7313076979922634
K: 15, MAE: 0.32653212012389526, RMSE: 0.6624886920529744
K: 20, MAE: 0.2729784936234361, RMSE: 0.6041453639780325
K: 25, MAE: 0.2384810026293552, RMSE: 0.5623611706205238
K: 30, MAE: 0.21304417013397986, RMSE: 0.5301509696579586
K: 35, MAE: 0.1990046999561297, RMSE: 0.5075051407373524
K: 40, MAE: 0.19339626496696558, RMSE: 0.4998279750307036
K: 45, MAE: 0.18830183279516122, RMSE: 0.4933867318482221
K: 50, MAE: 0.18051187166897656, RMSE: 0.48185642726737965
K: 55, MAE: 0.17860096842839532, RMSE: 0.47882626169927606
K: 60, MAE: 0.17567920897521475, RMSE: 0.47464018079039294
K: 65, MAE: 0.17544390091186166, RMSE: 0.4747571127671607
K: 70, MAE: 0.17428319061100217, RMSE: 0.4761039303805811
K: 75, MAE: 0.17450492250161745, RMSE: 0.4763131970745264
K: 80, MAE: 0.1740939953958801, RMSE: 0.4753024868701749
K: 85, MAE: 0.1726799779869552, RMSE: 0.47264623540045764
K: 90, MAE: 0.17175838077800432, RMSE: 0.47137328717520754
K: 95, MAE: 0.1

### Prediction with Suitable K Value

In [11]:
np_predictions_ead = np.zeros((n_users, n_items))

K = 95
EPSILON = 1e-9

# Precompute sums and counts for all users to avoid repeated computation
train_ds_sum = np.sum(train_ds, axis=1)
train_ds_count = np.clip(train_ds, 0, 1).sum(axis=1) + EPSILON

for (i, j), rating in np.ndenumerate(test_ds):
    if rating > 0:
        # Find top-k most similar users to the current user, excluding itself
        sim_user_ids = np.argsort(np_user_adjusted_euclidean_dist[i])[-(K + 1):-1]
        
        # The AED values of similar users
        sim_val = np_user_adjusted_euclidean_dist[i][sim_user_ids]
        
        # The similar users' ratings
        sim_users = test_ds[sim_user_ids]
        
        # Select the similar users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # If there is no similar users who have rated item s
        if not mask_rated_j.any():
            # It computes the mean value of ratings of current user on the rated items
            user_mean_rating = train_ds_sum[i] / train_ds_count[i]
            np_predictions_ead[i][j] = user_mean_rating
            np_predictions_ead[i][j] = np.clip(np_predictions_ead[i][j], 0, 5)
        
        # If there exist similar users who have rated item s
        else:
            # Initialize set and lists for users with dist(c, c') = 0
            sim_user_mask = sim_users[mask_rated_j]
            sim_val_mask = sim_val[mask_rated_j]
            C_cap_0 = np.where(sim_val_mask == 1)[0]
            
            # If the C_cap_0 has no elements
            if C_cap_0.size == 0:
                # Lets find the absolute value of each current, similar users pairs
                abs_sim_val = np.abs(sim_val_mask)
#                 print(f"sim_val_mask: {sim_val_mask}")
                # The sum of the absolute similarity values of each current, similar users pairs
                abs_sim_val_sum = np.sum(abs_sim_val)

                # Lets compute the value of k
                k = 1 / abs_sim_val_sum
                
                # The product of similarity and the rating of the similar_user on item j
                sv_r_cprime_s_prod = sim_val_mask * sim_user_mask[:, j]

                # The sum of product of similarity and rating of the similar_user on item j
                sv_r_cprime_s_prod_sum = np.sum(sv_r_cprime_s_prod)
                
                # Predictions using aed
                np_predictions_ead[i][j] = k * sv_r_cprime_s_prod_sum
                np_predictions_ead[i][j] = np.clip(np_predictions_ead[i][j], 0, 5)

            else:
                # Lets compute the number of co-rated items for each similar user
                # Which has the dist(c, c_cap) = 0
                m_ccap = np.array([
                    np.sum(np.logical_and(su > 0, test_ds[i] > 0))
                    for su in sim_user_mask[C_cap_0]
                ])

                # Calculates the rating of similar user on item j
                r_ccap_s = sim_user_mask[C_cap_0, j]
                
                # Calculates the sum of the ratings of similar users on item j
                m_ccap_sum = np.sum(m_ccap)
                
                # Computes the value for K note
                k_0 = 1 / m_ccap_sum

                # Computes the product of number of co-rated itmes for user c and ccap and 
                # the ratings of the corresponding ccap
                m_ccap_ccap_s_prod = m_ccap * r_ccap_s
                
                # The total sum of the product of m_ccap and r_ccap_s
                m_ccap_ccap_s_prod_sum = np.sum(m_ccap_ccap_s_prod)

                # the predicted rating
                np_predictions_ead[i][j] = k_0 * m_ccap_ccap_s_prod_sum
                np_predictions_ead[i][j] = np.clip(np_predictions_ead[i][j], 0, 5)

### Evaluation

In [12]:
evaluation_score = evaluate(test_ds, np_predictions_ead)

In [13]:
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 
# The Following MAE and RMSE is for Adjusted Euclidean Distance
# The value of K is set to 10

MAE = evaluation_score[0] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE = evaluation_score[1] # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.

In [14]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.1713535700910984, RMSE: 0.4687313902086791


In [15]:
# I have tried to implement as per the report research paper
# The implementation of equations are in the exact and 
# As mentioned concept in the research paper.

In [16]:
# Thanks and Regards